# Clean the Texas OIS dataset for analysis -- civilians shot data

* Inputs:
   * `OIS.xlsx`
   * `agencies_and_counties.csv` (used to add county information)
* Output: `shot_civilians.csv`

##### Author: Everett Wetchler (everett.wetchler@gmail.com)

## 1. Setup and read data

In [1]:
CLEANED_FILENAME = 'shot_civilians.csv'
DTW_OIS_PROJECT = 'tji/officer-involved-shootings'

In [2]:
import datadotworld as dw
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 100)

%load_ext watermark
%watermark -a "Everett Wetchler" -d -t -z -r -g -w -p datadotworld,numpy,pandas

Everett Wetchler 2018-05-25 14:36:55 CDT

datadotworld 1.6.0
numpy 1.14.3
pandas 0.22.0
Git hash: e7418abb91f2c2da17b31ea080238e52d84d260a
Git repo: git@github.com:texas-justice-initiative/data-processing.git
watermark 1.6.0


In [3]:
from lib.standardize_police_agency_names import standardize_agency_name
from lib.cleaning_tools import *

In [4]:
datasets = dw.load_dataset('tji/auxiliary-datasets', force_update=True)
agencies = datasets.dataframes['agencies_and_counties']

In [5]:
shootings = read_dtw_excel(DTW_OIS_PROJECT, 'original/OIS.xlsx', 'OISTable')
print("OIS civilians-shot incidents from %s to %s" % (
    shootings['Date of Incident'].min().strftime('%Y-%m-%d'),
    shootings['Date of Incident'].max().strftime('%Y-%m-%d')))
shootings.head()

Writing excel file to temp file: /var/folders/dc/8cbxbsh515s908xl0zyprszm0000gn/T/tmpyd42qvvd
OIS civilians-shot incidents from 2015-09-02 to 2018-05-14


,No.,Number of Reports Filed,Date of Report 1,Date AG Received,Name of Agency 1,City of Agency 1,Zip code of Agency 1,Date of Incident,Time of Incident,Name of Person 1 Filling out Form,Email Address of Person 1 Filling out Form,Date of Report 2,Name of Agency 2,City of Agency 2,Zip code of Agency 2,Name of Person 2 Filling out form,Email Address of Person 2 Filling out Form,Date of Report 3,Name of Agency 3,City of Agency 3,Zip code of Agency 3,Name of Person 3 Filling out form,Email Address of Person 3 Filling out Form,Date of Report 4,Name of Agency 4,City of Agency 4,Zip code of Agency 4,Name of Person 4 Filling out form,Email Address of Person 4 Filling out Form,Date of Report 5,Name of Agency 5,City of Agency 5,Zip code of Agency 5,Name of Person 5 Filling out form,Email Address of Person 5 Filling out Form,Date of Report 6,Name of Agency 6,City of Agency 6,Zip code of Agency 6,Name of Person 6 Filling out form,Email Address of Person 6 Filling out Form,Date of Report 7,Name of Agency 7,City of Agency 7,Zip code of Agency 7,Name of Person 7 Filling out form,Email Address of Person 7 Filling out Form,Date of Report 8,Name of Agency 8,City of Agency 8,...,Incident Resulted In,"Carried, Exhibited or Used Deadly Weapon",Peace Officer 1's Gender,Peace Officer 1's Age,Peace Officer 1's Race/Ethnicity,Peace Officer 2's Gender,Peace Officer 2's Age,Peace Officer 2's Race/Ethnicity,Peace Officer 3's Gender,Peace Officer 3's Age,Peace Officer 3's Race/Ethnicity,Peace Officer 4's Gender,Peace Officer 4's Age,Peace Officer 4's Race/Ethnicity,Peace Officer 5's Gender,Peace Officer 5's Age,Peace Officer 5's Race/Ethnicity,Peace Officer 6's Gender,Peace Officer 6's Age,Peace Officer 6's Race/Ethnicity,Peace Officer 7's Gender,Peace Officer 7's Age,Peace Officer 7's Race/Ethnicity,Peace Officer 8's Gender,Peace Officer 8's Age,Peace Officer 8's Race/Ethnicity,Peace Officer 9's Gender,Peace Officer 9's Age,Peace Officer 9's Race/Ethnicity,Peace Officer 10's Gender,Peace Officer 10's Age,Peace Officer 10's Race/Ethnicity,On Duty or Off Duty,Peace Officer Responding With 1 or More Officers,Incident Occurred During or as a Result of,Incident Occurred During or as a Result of 2,Incident Occurred During or as a Result of 3,"If Other, Specify Type of Call",Deadly Weapon Description,NEWS 1,NEWS 2,NEWS 3,NEWS 4,CDR?,CDR Narrative,Narrative Published by Law Enforcement,Column1,Column2,SHORTER,EXTRAS
0,1,1,9/16/2015,NaT,Freeport Police Department,Freeport,77541,2015-09-02,NaN,Pamela Morris,pmorris@freeport.tx.us,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Injury,No,Male,27,Hispanic or Latino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,On Duty,Yes,Traffic stop,NaN,NaN,Narcotic Stop and Evading Arrest,NaN,ABC 13,Your Southest Texas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,1,10/1/2015,NaT,Plano Police Department,Plano,75074,2015-09-03,NaN,Curtis Howard,curtish@plano.gov,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Injury,Yes,Male,30,Hispanic or Latino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,On Duty,No,Other - Specify type of call,NaN,NaN,Accidental discharge ricochet during range act...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,1,10/6/2015,NaT,Parker County Sheriff's Office,Weatherford,76086,2015-09-04,NaN,Meredith Gray,meredith.gray@parkercountytx.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Death,Yes,Male,27,Anglo or White,Male,56.0,Anglo or White,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,On Duty,Yes,Other - Speci

## 2. Begin cleaning

In [6]:
# Normalize column names
shootings.columns = [c.strip().lower() for c in shootings.columns]
# Normalize string cell values
upcase_strip_string_cells(shootings)
# Drop irrelevant columns
shootings.drop(['no.', 'column1', 'column2', 'extras'], axis=1, inplace=True)

In [7]:
# Make the column names more machine-friendly
col_renames = {
    "number of reports filed": "num_reports_filed",
    "date ag received": "date_ag_received",
    "date of incident": "date_incident",
    "time of incident": "time_incident",
    "injured or deceased's first name": "civilian_name_first",
    "injured or deceased's last name": "civilian_name_last",
    "injured or deceased's gender": "civilian_gender",
    "injured or deceased's age": "civilian_age",
    "injured or deceased's race/ethnicity": "civilian_race",
    "street address of incident": "incident_address",
    "city of incident": "incident_city",
    "county of incident": "incident_county",
    "zip code of incident": "incident_zip",
    "latitude of incident": "incident_latitude",
    "longitude of incident": "incident_longitude",
    "census tract of incident": "incident_census_tract",
    "incident resulted in": "incident_resulted_in",
    "carried, exhibited or used deadly weapon": "deadly_weapon",
    "on duty or off duty": "officer_on_duty",
    "peace officer responding with 1 or more officers": "multiple_officers_involved",
    "incident occurred during or as a result of": "incident_result_of",
    "if other, specify type of call": "incident_call_other",
    "deadly weapon description": "deadly_weapon_description",
    "cdr?": "custodial_death_report",
    "cdr narrative": "cdr_narrative",
    "narrative published by law enforcement": "lea_narrative_published",
    "shorter": "lea_narrative_shorter",
}

# Some columns have multiple copies for different individuals,
# e.g. agency_name_1 for the first officer's agency, then
# agency_name_2, for the second, etc. To avoid typing out
# all these numbers, we loop through such columns and rename
# them appropriately.
numerical_renames = {
    "date of report %d": "agency_report_date_%d",
    "name of agency %d": "agency_name_%d",
    "city of agency %d": "agency_city_%d",
    "zip code of agency %d": "agency_zip_%d",
    "name of person %d filling out form": "agency_name_person_filling_out_%d",
    "email address of person %d filling out form": "agency_email_person_filling_out_%d",
    "peace officer %d's gender": "officer_gender_%d",
    "peace officer %d's age": "officer_age_%d",
    "peace officer %d's race/ethnicity": "officer_race_%d",
    "incident occurred during or as a result of %d": "officer_caused_injury_%d",
    "news %d": "news_coverage_%d",
}
for i in range(1, 11):
    for k, v in numerical_renames.items():
        k = (k % i)
        v = v % i
        col_renames[k] = v

shootings.columns = [col_renames[c] for c in shootings.columns]
shootings.head()

,num_reports_filed,agency_report_date_1,date_ag_received,agency_name_1,agency_city_1,agency_zip_1,date_incident,time_incident,agency_name_person_filling_out_1,agency_email_person_filling_out_1,agency_report_date_2,agency_name_2,agency_city_2,agency_zip_2,agency_name_person_filling_out_2,agency_email_person_filling_out_2,agency_report_date_3,agency_name_3,agency_city_3,agency_zip_3,agency_name_person_filling_out_3,agency_email_person_filling_out_3,agency_report_date_4,agency_name_4,agency_city_4,agency_zip_4,agency_name_person_filling_out_4,agency_email_person_filling_out_4,agency_report_date_5,agency_name_5,agency_city_5,agency_zip_5,agency_name_person_filling_out_5,agency_email_person_filling_out_5,agency_report_date_6,agency_name_6,agency_city_6,agency_zip_6,agency_name_person_filling_out_6,agency_email_person_filling_out_6,agency_report_date_7,agency_name_7,agency_city_7,agency_zip_7,agency_name_person_filling_out_7,agency_email_person_filling_out_7,agency_report_date_8,agency_name_8,agency_city_8,agency_zip_8,...,incident_longitude,incident_latitude,incident_census_tract,incident_resulted_in,deadly_weapon,officer_gender_1,officer_age_1,officer_race_1,officer_gender_2,officer_age_2,officer_race_2,officer_gender_3,officer_age_3,officer_race_3,officer_gender_4,officer_age_4,officer_race_4,officer_gender_5,officer_age_5,officer_race_5,officer_gender_6,officer_age_6,officer_race_6,officer_gender_7,officer_age_7,officer_race_7,officer_gender_8,officer_age_8,officer_race_8,officer_gender_9,officer_age_9,officer_race_9,officer_gender_10,officer_age_10,officer_race_10,officer_on_duty,multiple_officers_involved,incident_result_of,officer_caused_injury_2,officer_caused_injury_3,incident_call_other,deadly_weapon_description,news_coverage_1,news_coverage_2,news_coverage_3,news_coverage_4,custodial_death_report,cdr_narrative,lea_narrative_published,lea_narrative_shorter
0,1,9/16/2015,NaT,FREEPORT POLICE DEPARTMENT,FREEPORT,77541,2015-09-02,NaN,PAMELA MORRIS,PMORRIS@FREEPORT.TX.US,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-95.356262,28.944891,664400.0,INJURY,NO,MALE,27,HISPANIC OR LATINO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ON DUTY,YES,TRAFFIC STOP,NaN,NaN,NARCOTIC STOP AND EVADING ARREST,NaN,ABC 13,YOUR SOUTHEST TEXAS,NaN,NaN,NaN,NaN,NaN,NaN
1,1,10/1/2015,NaT,PLANO POLICE DEPARTMENT,PLANO,75074,2015-09-03,NaN,CURTIS HOWARD,CURTISH@PLANO.GOV,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-96.642308,33.008128,32010.0,INJURY,YES,MALE,30,HISPANIC OR LATINO,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ON DUTY,NO,OTHER - SPECIFY TYPE OF CALL,NaN,NaN,ACCIDENTAL DISCHARGE RICOCHET DURING RANGE ACT...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1,10/6/2015,NaT,PARKER COUNTY SHERIFF'S OFFICE,WEATHERFORD,76086,2015-09-04,NaN,MEREDITH GRAY,MEREDITH.GRAY@PARKERCOUNTYTX.COM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,-97.634193,32.916724,140409.0,DEATH,YES,MALE,27,ANGLO OR WHITE,MALE,56.0,ANGLO OR WHITE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,ON DUTY,YES,OTHER - SPECIFY TYPE OF CALL,NaN,NaN,INVESTIGATION OF CRIMINAL ACTIVITY,FIREARM,WFAA,DFW CBS LOCAL,STAR TELEGRAM,FOX 4 NEWS,YES,DECEDENT SHOT A RIFLE AT LE OFFICERS WHOM RETU...,NaN,FIRED AT OFFICERS
3,1,9/11/2015,NaT,HOUSTON POLICE DEPARTMENT,HOUSTON,77002,2015-09-05,NaN,ODON BELMAREZ,ODON.BELMAREZ@HOUSTONPOLICE.ORG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,N

### Standardize common column types

In [8]:
standardize_gender_cols(shootings)
standardize_race_cols(shootings)
numericalize_age_cols(shootings)

### Create a column for the number of officers whose information was recorded here, `num_officers_recorded`

We use the presence of officer gender as an indicator. Not all incidents use a separate agency_name_X column per officer, so we can't use that.

In [9]:
officer_gender_cols = ['officer_gender_%d' % i for i in range(1, 11)]
shootings['num_officers_recorded'] = shootings[officer_gender_cols].notnull().sum(axis=1)
shootings[shootings['num_officers_recorded'] > 5].head()

,num_reports_filed,agency_report_date_1,date_ag_received,agency_name_1,agency_city_1,agency_zip_1,date_incident,time_incident,agency_name_person_filling_out_1,agency_email_person_filling_out_1,agency_report_date_2,agency_name_2,agency_city_2,agency_zip_2,agency_name_person_filling_out_2,agency_email_person_filling_out_2,agency_report_date_3,agency_name_3,agency_city_3,agency_zip_3,agency_name_person_filling_out_3,agency_email_person_filling_out_3,agency_report_date_4,agency_name_4,agency_city_4,agency_zip_4,agency_name_person_filling_out_4,agency_email_person_filling_out_4,agency_report_date_5,agency_name_5,agency_city_5,agency_zip_5,agency_name_person_filling_out_5,agency_email_person_filling_out_5,agency_report_date_6,agency_name_6,agency_city_6,agency_zip_6,agency_name_person_filling_out_6,agency_email_person_filling_out_6,agency_report_date_7,agency_name_7,agency_city_7,agency_zip_7,agency_name_person_filling_out_7,agency_email_person_filling_out_7,agency_report_date_8,agency_name_8,agency_city_8,agency_zip_8,...,incident_latitude,incident_census_tract,incident_resulted_in,deadly_weapon,officer_gender_1,officer_age_1,officer_race_1,officer_gender_2,officer_age_2,officer_race_2,officer_gender_3,officer_age_3,officer_race_3,officer_gender_4,officer_age_4,officer_race_4,officer_gender_5,officer_age_5,officer_race_5,officer_gender_6,officer_age_6,officer_race_6,officer_gender_7,officer_age_7,officer_race_7,officer_gender_8,officer_age_8,officer_race_8,officer_gender_9,officer_age_9,officer_race_9,officer_gender_10,officer_age_10,officer_race_10,officer_on_duty,multiple_officers_involved,incident_result_of,officer_caused_injury_2,officer_caused_injury_3,incident_call_other,deadly_weapon_description,news_coverage_1,news_coverage_2,news_coverage_3,news_coverage_4,custodial_death_report,cdr_narrative,lea_narrative_published,lea_narrative_shorter,num_officers_recorded
62,8,1/5/2016,NaT,ODESSA POLICE DEPARTMENT,ODESSA,78761,2015-12-23,NaN,DAVID LARA,DLARA@ODESSA-TX.GOV,1/5/2016,ODESSA POLICE DEPARTMENT,ODESSA,79761,DAVID LARA,DLARA@ODESSA-TX.GOV,1/5/2016,ODESSA POLICE DEPARTMENT,ODESSA,79761,DAVID LARA,DLARA@ODESSA-TX.GOV,1/5/2016,ODESSA POLICE DEPARTMENT,ODESSA,79761,DAVID LARA,DLARA@ODESSA-TX.GOV,1/5/2016,ODESSA POLICE DEPARTMENT,ODESSA,79761,DAVID LARA,DLARA@ODESSA-TX.GOV,1/5/2016,ODESSA POLICE DEPARTMENT,ODESSA,79761.0,DAVID LARA,DLARA@ODESSA-TX.GOV,1/5/2016,ODESSA POLICE DEPARTMENT,ODESSA,79761.0,DAVID LARA,DLARA@ODESSA-TX.GOV,1/5/2016,ODESSA POLICE DEPARTMENT,ODESSA,79761.0,...,31.865143,1700.0,INJURY,YES,M,26.0,HISPANIC,M,26.0,WHITE,M,40.0,WHITE,M,24.0,WHITE,M,28.0,HISPANIC,M,42.0,HISPANIC,M,27.0,HISPANIC,M,41.0,HISPANIC,NaN,NaN,NaN,NaN,NaN,NaN,ON DUTY,YES,EXECUTION OF A WARRANT,NaN,NaN,NaN,FIREARM,MY SAN ANTONIO,NEWS WEST 9,CBS 7,KSAT,NaN,NaN,NaN,NaN,8
157,10,2/23/2017,2017-02-23,DART POLICE DEPARTMENT,DALLAS,75203,2016-07-07,21:00:00,LT. SHERRI PLUNK #43,SPLUNK@DART.ORG,2/23/2017,DART POLICE DEPARTMENT,DALLAS,75203,LT. SHERRI PLUNK #43,SPLUNK@DART.ORG,2/23/2017,DART POLICE DEPARTMENT,DALLAS,75203,LT. SHERRI PLUNK #43,SPLUNK@DART.ORG,2/23/2017,DART POLICE DEPARTMENT,DALLAS,75203,LT. SHERRI PLUNK #43,SPLUNK@DART.ORG,2/7/2017,DALLAS POLICE DEPARTMENT,DALLAS,75215,SGT. E. MERRITT #8112,E.MERRITT@DPD.DALLASCITYHALL.COM,2/7/2017,DALLAS POLICE DEPARTMENT,DALLAS,75215.0,SGT. E. MERRITT #8112,E.MERRITT@DPD.DALLASCITYHALL.COM,2/7/2017,DALLAS POLICE DEPARTMENT,DALLAS,75215.0,SGT. E. MERRITT #8112,E.MERRITT@DPD.DALLASCITYHALL.COM,2/7/2017,DALLAS POLICE DEPARTMENT,DALLAS,75215.0,...,32.779788,3101.0,DEATH,YES,M,63.0,WHITE,M,44.0,WHITE,M,43.0,WHITE,M,37.0,BLACK,M,27.0,WHITE,M,31.0,HISPANIC,M,34.0,OTHER,M,39.0,WHITE,M,41.0,WHITE,M,46.0,WHITE,ON DUTY,YES,OTHER - SPECIFY TYPE OF CALL,NaN,NaN,AMBUSH OF OFFICERS BY SUSPECT. WOUNDED ON 7/7/...,NaN,DPD PRESS RELEASE,NaN,NaN,NaN,YES,"ON JULY 7, 2016, AT APPROXIMATELY 8:57 P.M., D...",NaN,FIRED AT OFFICERS,10
271,8,3/15/2017,2017-03-15,CLUTE POLICE DEPARTMENT,CLUTE,77531,2017-02-24,14:20:00,CHIEF

In [10]:
shootings['custodial_death_report'] = shootings['custodial_death_report'].fillna('n').apply(lambda c: c.strip().lower()[0] == 'y')
shootings['multiple_officers_involved'] = shootings['multiple_officers_involved'].fillna('n').apply(lambda c: c.strip().lower()[0] == 'y')

shootings['civilian_died'] = shootings['incident_resulted_in'].apply(lambda x: x.strip().lower()) == 'death'
shootings.drop('incident_resulted_in', axis=1, inplace=True)
shootings['incident_result_of'] = shootings['incident_result_of'].apply(lambda x: x.strip().lower())

shootings['officer_on_duty'] = shootings['officer_on_duty'].apply(lambda x: x if pd.isnull(x) else (x.strip().lower().startswith('on ')))
shootings['deadly_weapon_description'] = shootings['deadly_weapon_description'].apply(lambda w: w if pd.isnull(w) else w.strip().lower())

### NOTE: Data quirk

It's unclear how many officers were actually at the scene
* The `multiple_officer_involved` column is a yes/no column, but there are also columns to list the agency, gender, etc for each officer. These do not always agree. Sometimes `multiple_officer_involved` is yes, but only one officer's details are recorded. And sometimes we have details for many officers, but `multiple_officer_involved` is no. See below.
* The punchline is to interpret these columns with caution

In [11]:
pd.crosstab(shootings.multiple_officers_involved, shootings.num_officers_recorded)

num_officers_recorded,1,2,3,4,5,6,7,8,10
multiple_officers_involved,,,,,,,,,
False,79,2,2,0,1,1,0,0,0
True,240,78,28,18,9,1,4,2,1


### Handle weapons-related questions

In [12]:
# Convert yes/no to boolean
shootings['deadly_weapon'] = shootings['deadly_weapon'].apply(lambda x: x if pd.isnull(x) else (x.strip().lower() == 'yes'))
shootings['deadly_weapon'].value_counts()

True     387
False     79
Name: deadly_weapon, dtype: int64

In [13]:
# Check for mistakes. Sometimes the "was there a deadly weapon?" question
# is answered with "No" while a description of a deadly weapon is given.
pd.crosstab(shootings['deadly_weapon'], shootings['deadly_weapon_description'].notnull())

deadly_weapon_description,False,True
deadly_weapon,,
False,64,15
True,59,328


In [14]:
# Let's look at these instances to be sure
s = shootings[~shootings['deadly_weapon'] & pd.notnull(shootings['deadly_weapon_description'])][['deadly_weapon', 'deadly_weapon_description']]
print(len(s))
s

15


,deadly_weapon,deadly_weapon_description
40,False,vehicle
56,False,firearm
162,False,bb gun
227,False,vehicle
252,False,vehicle
268,False,vehicle
286,False,vehicle
300,False,vehicle
321,False,took officer's taser
323,False,fell while getting out of car?


In [15]:
# It seems clear that we should correct the 'deadly_weapon' category
# in these cases.
#
# For cases where the user said there WAS a deadly weapon,
# but did not give a weapon description, we'll assume there really
# was a weapon but use a special category (see cells below).
shootings['deadly_weapon'] = (shootings['deadly_weapon'] | pd.notnull(shootings['deadly_weapon_description']))
shootings['deadly_weapon'].value_counts()

True     402
False     64
Name: deadly_weapon, dtype: int64

In [16]:
# What weapons to people use? Can we categorize them?
sorted(list(x for x in set(shootings.deadly_weapon_description) if pd.notnull(x)))

['"agent was assaulted"',
 'air soft gun',
 'armed',
 'arms (choking)',
 'assault rifle',
 'axe',
 'baseball bat',
 'baseball bat and fireplace poker',
 'bb gun',
 'body',
 'bomb',
 'box cutter',
 'butcher knife',
 'car',
 'club, bat',
 "deputy's gun",
 'fell while getting out of car?',
 'firearm',
 'glock 40',
 'gun',
 'handgun',
 'hatchet',
 'imitation weapon',
 'knife',
 'knife - not opened',
 'knife, gun',
 'knives',
 'long gun',
 'machete',
 'metal flashlight',
 'pellet gun',
 'pencil',
 'pickaxe',
 'pistol',
 'reports say unarmed',
 'revolver',
 'rifle',
 'rock',
 'sawed-off shotgun',
 'scissors, screwdriver',
 'semi-automatic rifle',
 'sharp metal object (piece of headphones)',
 'shotgun',
 'sword',
 "took officer's knife",
 "took officer's taser",
 'truck',
 'vehicle',
 'weapon',
 'weed-cutter']

In [17]:
# Manual categorization of weapons
weapon_types = {
    'FIREARM': ['handgun', 'sawed-off shotgun', 'revolver', 'rifle',
                'assault rifle', 'firearm', 'shotgun', 'long gun', 'gun',
                'glock 40', 'pistol', 'knife, gun', "deputy's gun"],
    'KNIFE/CUTTING': ['hatchet', 'butcher knife', 'knife', 'knives', 'box cutter',
                'knife - not opened', 'machete', 'sword', 'axe'],
    'VEHICLE': ['car', 'truck', 'vehicle'],
}
type_lookup = {}
for k, v in weapon_types.items():
    for w in v:
        if w in type_lookup:
            print("DUPLICATE:", k, w)
        type_lookup[w] = k

weapons = []
for has_weapon, desc in zip(shootings['deadly_weapon'], shootings['deadly_weapon_description']):
    if pd.isnull(desc) or not desc:
        if has_weapon:
            weapons.append('(DETAILS MISSING)')
        else:
            weapons.append(np.nan)
        continue
    weapons.append(type_lookup.get(desc, 'OTHER'))

shootings['deadly_weapon_category'] = weapons    
shootings['deadly_weapon_category'].value_counts()

FIREARM              228
(DETAILS MISSING)     59
KNIFE/CUTTING         48
OTHER                 34
VEHICLE               33
Name: deadly_weapon_category, dtype: int64

In [18]:
sorted(list(shootings['deadly_weapon_description'][shootings['deadly_weapon_category'] == 'OTHER']))

['"agent was assaulted"',
 'air soft gun',
 'armed',
 'arms (choking)',
 'baseball bat',
 'baseball bat and fireplace poker',
 'bb gun',
 'bb gun',
 'bb gun',
 'bb gun',
 'body',
 'body',
 'bomb',
 'club, bat',
 'fell while getting out of car?',
 'imitation weapon',
 'metal flashlight',
 'pellet gun',
 'pellet gun',
 'pencil',
 'pickaxe',
 'reports say unarmed',
 'rock',
 'scissors, screwdriver',
 'semi-automatic rifle',
 'semi-automatic rifle',
 'sharp metal object (piece of headphones)',
 "took officer's knife",
 "took officer's taser",
 "took officer's taser",
 'weapon',
 'weapon',
 'weapon',
 'weed-cutter']

### Uppercase the content of all columns

In [19]:
print('Uppercasing columns: ', end='')
for col, dt in shootings.dtypes.items():
    if dt == 'object':
        print(col, end=' ')
        shootings[col] = shootings[col].apply(lambda s: s if type(s) != str else s.upper())

Uppercasing columns: agency_report_date_1 agency_name_1 agency_city_1 agency_zip_1 time_incident agency_name_person_filling_out_1 agency_email_person_filling_out_1 agency_report_date_2 agency_name_2 agency_city_2 agency_zip_2 agency_name_person_filling_out_2 agency_email_person_filling_out_2 agency_report_date_3 agency_name_3 agency_city_3 agency_zip_3 agency_name_person_filling_out_3 agency_email_person_filling_out_3 agency_report_date_4 agency_name_4 agency_city_4 agency_zip_4 agency_name_person_filling_out_4 agency_email_person_filling_out_4 agency_report_date_5 agency_name_5 agency_city_5 agency_zip_5 agency_name_person_filling_out_5 agency_email_person_filling_out_5 agency_report_date_6 agency_name_6 agency_city_6 agency_name_person_filling_out_6 agency_email_person_filling_out_6 agency_report_date_7 agency_name_7 agency_city_7 agency_name_person_filling_out_7 agency_email_person_filling_out_7 agency_report_date_8 agency_name_8 agency_city_8 agency_name_person_filling_out_8 agency

### Standardize police agency names

In [20]:
for i in range(1, 11):
    shootings['agency_name_%d' % i] = shootings['agency_name_%d' % i].apply(standardize_agency_name)

shootings.head()

,num_reports_filed,agency_report_date_1,date_ag_received,agency_name_1,agency_city_1,agency_zip_1,date_incident,time_incident,agency_name_person_filling_out_1,agency_email_person_filling_out_1,agency_report_date_2,agency_name_2,agency_city_2,agency_zip_2,agency_name_person_filling_out_2,agency_email_person_filling_out_2,agency_report_date_3,agency_name_3,agency_city_3,agency_zip_3,agency_name_person_filling_out_3,agency_email_person_filling_out_3,agency_report_date_4,agency_name_4,agency_city_4,agency_zip_4,agency_name_person_filling_out_4,agency_email_person_filling_out_4,agency_report_date_5,agency_name_5,agency_city_5,agency_zip_5,agency_name_person_filling_out_5,agency_email_person_filling_out_5,agency_report_date_6,agency_name_6,agency_city_6,agency_zip_6,agency_name_person_filling_out_6,agency_email_person_filling_out_6,agency_report_date_7,agency_name_7,agency_city_7,agency_zip_7,agency_name_person_filling_out_7,agency_email_person_filling_out_7,agency_report_date_8,agency_name_8,agency_city_8,agency_zip_8,...,incident_census_tract,deadly_weapon,officer_gender_1,officer_age_1,officer_race_1,officer_gender_2,officer_age_2,officer_race_2,officer_gender_3,officer_age_3,officer_race_3,officer_gender_4,officer_age_4,officer_race_4,officer_gender_5,officer_age_5,officer_race_5,officer_gender_6,officer_age_6,officer_race_6,officer_gender_7,officer_age_7,officer_race_7,officer_gender_8,officer_age_8,officer_race_8,officer_gender_9,officer_age_9,officer_race_9,officer_gender_10,officer_age_10,officer_race_10,officer_on_duty,multiple_officers_involved,incident_result_of,officer_caused_injury_2,officer_caused_injury_3,incident_call_other,deadly_weapon_description,news_coverage_1,news_coverage_2,news_coverage_3,news_coverage_4,custodial_death_report,cdr_narrative,lea_narrative_published,lea_narrative_shorter,num_officers_recorded,civilian_died,deadly_weapon_category
0,1,9/16/2015,NaT,FREEPORT POLICE DEPT,FREEPORT,77541,2015-09-02,NaN,PAMELA MORRIS,PMORRIS@FREEPORT.TX.US,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,...,664400.0,False,M,27.0,HISPANIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,TRAFFIC STOP,NaN,NaN,NARCOTIC STOP AND EVADING ARREST,NaN,ABC 13,YOUR SOUTHEST TEXAS,NaN,NaN,False,NaN,NaN,NaN,1,False,NaN
1,1,10/1/2015,NaT,PLANO POLICE DEPT,PLANO,75074,2015-09-03,NaN,CURTIS HOWARD,CURTISH@PLANO.GOV,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,...,32010.0,True,M,30.0,HISPANIC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,False,OTHER - SPECIFY TYPE OF CALL,NaN,NaN,ACCIDENTAL DISCHARGE RICOCHET DURING RANGE ACT...,NaN,NaN,NaN,NaN,NaN,False,NaN,NaN,NaN,1,False,(DETAILS MISSING)
2,1,10/6/2015,NaT,PARKER CO SHERIFFS OFFICE,WEATHERFORD,76086,2015-09-04,NaN,MEREDITH GRAY,MEREDITH.GRAY@PARKERCOUNTYTX.COM,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,...,140409.0,True,M,27.0,WHITE,M,56.0,WHITE,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,True,OTHER - SPECIFY TYPE OF CALL,NaN,NaN,INVESTIGATION OF CRIMINAL ACTIVITY,FIREARM,WFAA,DFW CBS LOCAL,STAR TELEGRAM,FOX 4 NEWS,True,DECEDENT SHOT A RIFLE AT LE OFFICERS WHOM RETU...,NaN,FIRED AT OFFICERS,2,True,FIREARM
3,1,9/11/2015,NaT,HOUSTON POLICE DEPT,HOUSTON,77002,2015-09-05,NaN,ODON BELMAREZ,ODON.BELMAREZ@HOUSTONPOLICE.ORG,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,...,332100.0,True,M,28.0,WHITE,NaN,NaN,NaN,NaN,NaN,NaN,

### Add county information

In [21]:
dept_to_county = dict(zip(agencies.agency, agencies.county))
for i in range(1, 11):
    shootings = insert_col_after(
        shootings,
        shootings['agency_name_%d' % i].apply(lambda d: dept_to_county.get(d, np.nan)),
        'agency_county_%d' % i,
        'agency_city_%d' % i)

shootings.agency_county_1.isnull().value_counts()

False    462
True       4
Name: agency_county_1, dtype: int64

In [22]:
shootings[shootings.agency_county_1.isnull()].agency_name_1.tolist()

['JAL POLICE DEPT',
 'DART POLICE DEPT',
 'DRUG ENFORCEMENT ADMINISTRATION US DOJ',
 'TEXAS DEPT OF PUBLIC SAFETY CRIMINAL INVESTIGATIONS DIVISION']

### Other analysis revealed some typos with agency name. We'll demonstrate them here before correcting.

In [23]:
tmp = shootings.groupby(['incident_county', 'incident_city']).size().sort_values().unstack().T
tmax = tmp.max()
county_to_biggest_city = {}
for c in tmp.columns:
    x = tmp[c][tmp[c] == tmax[c]]
    county_to_biggest_city[c] = x.index[0]

In [24]:
TOP5 = list(shootings.incident_county.value_counts().head(5).index)
TOP5_CITIES = [county_to_biggest_city.get(c) for c in TOP5]
print(TOP5)
print(TOP5_CITIES)

['HARRIS', 'BEXAR', 'DALLAS', 'TARRANT', 'TRAVIS']
['HOUSTON', 'SAN ANTONIO', 'DALLAS', 'FORT WORTH', 'AUSTIN']


In [25]:
for county in TOP5:
    print("-- %s --" % county)
    print(shootings[shootings.incident_county == county]['agency_county_1'].value_counts())
    print()

-- HARRIS --
HARRIS        93
BRAZORIA       1
MONTGOMERY     1
WALLER         1
STATE          1
Name: agency_county_1, dtype: int64

-- BEXAR --
BEXAR     40
STATE      5
WILSON     1
Name: agency_county_1, dtype: int64

-- DALLAS --
DALLAS     38
KAUFMAN     1
STATE       1
Name: agency_county_1, dtype: int64

-- TARRANT --
TARRANT    34
PARKER      1
DALLAS      1
Name: agency_county_1, dtype: int64

-- TRAVIS --
TRAVIS    21
Name: agency_county_1, dtype: int64



### 'HARRIS' and 'HARRISON' counties are nowhere near each other, so it seems more likely that there is a typo or data error than that four incidents in HARRIS county involved officers from HARRISON county. Let's check them out.

In [26]:
shootings[(shootings.incident_county == 'HARRIS') & (shootings.agency_county_1 == 'HARRISON')]

,num_reports_filed,agency_report_date_1,date_ag_received,agency_name_1,agency_city_1,agency_county_1,agency_zip_1,date_incident,time_incident,agency_name_person_filling_out_1,agency_email_person_filling_out_1,agency_report_date_2,agency_name_2,agency_city_2,agency_county_2,agency_zip_2,agency_name_person_filling_out_2,agency_email_person_filling_out_2,agency_report_date_3,agency_name_3,agency_city_3,agency_county_3,agency_zip_3,agency_name_person_filling_out_3,agency_email_person_filling_out_3,agency_report_date_4,agency_name_4,agency_city_4,agency_county_4,agency_zip_4,agency_name_person_filling_out_4,agency_email_person_filling_out_4,agency_report_date_5,agency_name_5,agency_city_5,agency_county_5,agency_zip_5,agency_name_person_filling_out_5,agency_email_person_filling_out_5,agency_report_date_6,agency_name_6,agency_city_6,agency_county_6,agency_zip_6,agency_name_person_filling_out_6,agency_email_person_filling_out_6,agency_report_date_7,agency_name_7,agency_city_7,agency_county_7,...,incident_census_tract,deadly_weapon,officer_gender_1,officer_age_1,officer_race_1,officer_gender_2,officer_age_2,officer_race_2,officer_gender_3,officer_age_3,officer_race_3,officer_gender_4,officer_age_4,officer_race_4,officer_gender_5,officer_age_5,officer_race_5,officer_gender_6,officer_age_6,officer_race_6,officer_gender_7,officer_age_7,officer_race_7,officer_gender_8,officer_age_8,officer_race_8,officer_gender_9,officer_age_9,officer_race_9,officer_gender_10,officer_age_10,officer_race_10,officer_on_duty,multiple_officers_involved,incident_result_of,officer_caused_injury_2,officer_caused_injury_3,incident_call_other,deadly_weapon_description,news_coverage_1,news_coverage_2,news_coverage_3,news_coverage_4,custodial_death_report,cdr_narrative,lea_narrative_published,lea_narrative_shorter,num_officers_recorded,civilian_died,deadly_weapon_category


### Yup. These are officers from Baytown and Spring, which border Houston (Harris, not Harrison county). Their emails are also from `hctx.net` which is Harris County. Their given agency names must be errors. 

In [27]:
city_county_corrections = [
    ['BAYTOWN', 'HARRISON', 'HARRIS'],
    ['SPRING', 'HARRISON', 'HARRIS'],
]
corrected = set()
for city, wrong_county, right_county in city_county_corrections:
    for i in range(1, 11):
        tmp = shootings[(shootings['agency_city_%d' % i] == city) & (shootings['agency_county_%d' % i] == wrong_county)]
        if len(tmp) == 0:
            break
        shootings.loc[tmp.index, 'agency_county_%d' % i] = right_county
        for idx, name in tmp['agency_name_%d' % i].items():
            shootings.loc[idx, 'agency_name_%d' % i] = name.replace(wrong_county, right_county)
            corrected.add(idx)

shootings.loc[corrected]

,num_reports_filed,agency_report_date_1,date_ag_received,agency_name_1,agency_city_1,agency_county_1,agency_zip_1,date_incident,time_incident,agency_name_person_filling_out_1,agency_email_person_filling_out_1,agency_report_date_2,agency_name_2,agency_city_2,agency_county_2,agency_zip_2,agency_name_person_filling_out_2,agency_email_person_filling_out_2,agency_report_date_3,agency_name_3,agency_city_3,agency_county_3,agency_zip_3,agency_name_person_filling_out_3,agency_email_person_filling_out_3,agency_report_date_4,agency_name_4,agency_city_4,agency_county_4,agency_zip_4,agency_name_person_filling_out_4,agency_email_person_filling_out_4,agency_report_date_5,agency_name_5,agency_city_5,agency_county_5,agency_zip_5,agency_name_person_filling_out_5,agency_email_person_filling_out_5,agency_report_date_6,agency_name_6,agency_city_6,agency_county_6,agency_zip_6,agency_name_person_filling_out_6,agency_email_person_filling_out_6,agency_report_date_7,agency_name_7,agency_city_7,agency_county_7,...,incident_census_tract,deadly_weapon,officer_gender_1,officer_age_1,officer_race_1,officer_gender_2,officer_age_2,officer_race_2,officer_gender_3,officer_age_3,officer_race_3,officer_gender_4,officer_age_4,officer_race_4,officer_gender_5,officer_age_5,officer_race_5,officer_gender_6,officer_age_6,officer_race_6,officer_gender_7,officer_age_7,officer_race_7,officer_gender_8,officer_age_8,officer_race_8,officer_gender_9,officer_age_9,officer_race_9,officer_gender_10,officer_age_10,officer_race_10,officer_on_duty,multiple_officers_involved,incident_result_of,officer_caused_injury_2,officer_caused_injury_3,incident_call_other,deadly_weapon_description,news_coverage_1,news_coverage_2,news_coverage_3,news_coverage_4,custodial_death_report,cdr_narrative,lea_narrative_published,lea_narrative_shorter,num_officers_recorded,civilian_died,deadly_weapon_category


### While we're at it, are there any mistakes the other way? (Harris county officers showing up in Harrison county)
#### Answer: no.

In [28]:
shootings[(shootings.incident_county == 'HARRISON') & (shootings.agency_county_1 == 'HARRIS')]

,num_reports_filed,agency_report_date_1,date_ag_received,agency_name_1,agency_city_1,agency_county_1,agency_zip_1,date_incident,time_incident,agency_name_person_filling_out_1,agency_email_person_filling_out_1,agency_report_date_2,agency_name_2,agency_city_2,agency_county_2,agency_zip_2,agency_name_person_filling_out_2,agency_email_person_filling_out_2,agency_report_date_3,agency_name_3,agency_city_3,agency_county_3,agency_zip_3,agency_name_person_filling_out_3,agency_email_person_filling_out_3,agency_report_date_4,agency_name_4,agency_city_4,agency_county_4,agency_zip_4,agency_name_person_filling_out_4,agency_email_person_filling_out_4,agency_report_date_5,agency_name_5,agency_city_5,agency_county_5,agency_zip_5,agency_name_person_filling_out_5,agency_email_person_filling_out_5,agency_report_date_6,agency_name_6,agency_city_6,agency_county_6,agency_zip_6,agency_name_person_filling_out_6,agency_email_person_filling_out_6,agency_report_date_7,agency_name_7,agency_city_7,agency_county_7,...,incident_census_tract,deadly_weapon,officer_gender_1,officer_age_1,officer_race_1,officer_gender_2,officer_age_2,officer_race_2,officer_gender_3,officer_age_3,officer_race_3,officer_gender_4,officer_age_4,officer_race_4,officer_gender_5,officer_age_5,officer_race_5,officer_gender_6,officer_age_6,officer_race_6,officer_gender_7,officer_age_7,officer_race_7,officer_gender_8,officer_age_8,officer_race_8,officer_gender_9,officer_age_9,officer_race_9,officer_gender_10,officer_age_10,officer_race_10,officer_on_duty,multiple_officers_involved,incident_result_of,officer_caused_injury_2,officer_caused_injury_3,incident_call_other,deadly_weapon_description,news_coverage_1,news_coverage_2,news_coverage_3,news_coverage_4,custodial_death_report,cdr_narrative,lea_narrative_published,lea_narrative_shorter,num_officers_recorded,civilian_died,deadly_weapon_category


### Flag records that are part of the same incident

In [29]:
incident_records = shootings.groupby(['incident_address', 'date_incident']).size().sort_values(ascending=False).to_dict()
num_records = []
for address, date in zip(shootings.incident_address, shootings.date_incident):
    if pd.notnull(address) and pd.notnull(date):
        num_records.append(incident_records[(address, date)])
    else:
        num_records.append(1)
# TODO(wetchler) -- add this step to the data cleaning
shootings['num_rows_about_this_incident'] = num_records

### Add a column for full name

In [30]:
s = shootings.civilian_name_first.fillna('') + ' ' + shootings.civilian_name_last.fillna('')
s = s.apply(lambda s: ' '.join(s.strip().split()))
s[s == ''] = np.nan
shootings = insert_col_after(shootings, s, 'civilian_name_full', 'civilian_name_last')

### Re-order columns more sensibly
* **Disclaimer**: This code is gross and probably an OCD attempt to make the final result look nice. Just destroy this cell if it proves too confusing or opaque.

In [31]:
section_order = ['date', 'incident', 'civilian', 'agency', 'news']
agency_fields = ['name', 'report_date', 'city', 'county', 'zip', 'name_person_filling_out', 'email_person_filling_out']
officer_fields = ['age', 'race', 'gender', 'on_duty', 'caused_injury']
new_col_order = []
remaining = set(shootings.columns)
for s in section_order:
    if s == 'agency':
        for i in range(1, 11):
            for field in officer_fields:
                c = 'officer_%s_%d' % (field, i)
                if c in remaining:
                    new_col_order.append(c)
                    remaining.remove(c)
            for field in agency_fields:
                c = 'agency_%s_%d' % (field, i)
                new_col_order.append(c)
                remaining.remove(c)
    else:
        this_section_cols = []
        for c in shootings.columns:
            if c in remaining and c.startswith(s + '_'):
                this_section_cols.append(c)
                remaining.remove(c)
        new_col_order.extend(this_section_cols)

new_col_order.extend(sorted(remaining))
shape_before = shootings.shape
shootings = shootings[new_col_order]
assert(shootings.shape == shape_before)

In [32]:
shootings.sort_values(['date_incident', 'incident_county', 'agency_county_1'], inplace=True)
shootings.head(100)

,date_ag_received,date_incident,incident_address,incident_city,incident_county,incident_zip,incident_longitude,incident_latitude,incident_census_tract,incident_result_of,incident_call_other,civilian_name_first,civilian_name_last,civilian_name_full,civilian_gender,civilian_age,civilian_race,civilian_died,officer_age_1,officer_race_1,officer_gender_1,agency_name_1,agency_report_date_1,agency_city_1,agency_county_1,agency_zip_1,agency_name_person_filling_out_1,agency_email_person_filling_out_1,officer_age_2,officer_race_2,officer_gender_2,officer_caused_injury_2,agency_name_2,agency_report_date_2,agency_city_2,agency_county_2,agency_zip_2,agency_name_person_filling_out_2,agency_email_person_filling_out_2,officer_age_3,officer_race_3,officer_gender_3,officer_caused_injury_3,agency_name_3,agency_report_date_3,agency_city_3,agency_county_3,agency_zip_3,agency_name_person_filling_out_3,agency_email_person_filling_out_3,...,agency_zip_7,agency_name_person_filling_out_7,agency_email_person_filling_out_7,officer_age_8,officer_race_8,officer_gender_8,agency_name_8,agency_report_date_8,agency_city_8,agency_county_8,agency_zip_8,agency_name_person_filling_out_8,agency_email_person_filling_out_8,officer_age_9,officer_race_9,officer_gender_9,agency_name_9,agency_report_date_9,agency_city_9,agency_county_9,agency_zip_9,agency_name_person_filling_out_9,agency_email_person_filling_out_9,officer_age_10,officer_race_10,officer_gender_10,agency_name_10,agency_report_date_10,agency_city_10,agency_county_10,agency_zip_10,agency_name_person_filling_out_10,agency_email_person_filling_out_10,news_coverage_1,news_coverage_2,news_coverage_3,news_coverage_4,cdr_narrative,custodial_death_report,deadly_weapon,deadly_weapon_category,deadly_weapon_description,lea_narrative_published,lea_narrative_shorter,multiple_officers_involved,num_officers_recorded,num_reports_filed,num_rows_about_this_incident,officer_on_duty,time_incident
0,NaT,2015-09-02,1010 MAGNOLIA STREET,FREEPORT,BRAZORIA,77541.0,-95.356262,28.944891,664400.0,TRAFFIC STOP,NARCOTIC STOP AND EVADING ARREST,RICKEY,MAYBERRY,RICKEY MAYBERRY,M,30.0,BLACK,False,27.0,HISPANIC,M,FREEPORT POLICE DEPT,9/16/2015,FREEPORT,BRAZORIA,77541,PAMELA MORRIS,PMORRIS@FREEPORT.TX.US,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,ABC 13,YOUR SOUTHEST TEXAS,NaN,NaN,NaN,False,False,NaN,NaN,NaN,NaN,True,1,1,1,True,NaN
1,NaT,2015-09-03,4840 E. PLANO PARKWAY,PLANO,COLLIN,75074.0,-96.642308,33.008128,32010.0,OTHER - SPECIFY TYPE OF CALL,ACCIDENTAL DISCHARGE RICOCHET DURING RANGE ACT...,NaN,NaN,NaN,M,55.0,WHITE,False,30.0,HISPANIC,M,PLANO POLICE DEPT,10/1/2015,PLANO,COLLIN,75074,CURTIS HOWARD,CURTISH@PLANO.GOV,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False,True,(DETAILS MISSING),NaN,NaN,NaN,False,1,1,1,True,NaN
2,NaT,2015-09-04,101 COUCH CT.,SPRINGTOWN,PARKER,76082.0,-97.634193,32.916724,140409.0,OTHER - SPECIFY TYPE OF CALL,INVESTIGATION OF CRIMINAL ACTIVITY,SULLY JOE,LANIER,SULLY JOE LANIER,M,36.0,WHITE,True,27.0,WHITE,M,PARKER CO SHERIFFS OFFICE,10/6/2015,WEATHERFORD,PARKER,76086,MEREDITH GRAY,MEREDITH.GRAY@PARKERCOUNTYTX.COM,56.0,WHITE,M,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,WFAA,DFW CBS LOCAL,STAR TELEGRAM,FOX 4 NEWS,DECEDENT SHOT A RIFLE AT LE OFFICERS WHOM RETU...,True,True,FIREARM,FIREARM,NaN,FIRED AT OFFICERS,True,2,1,1,True,NaN
3,NaT,2015-09-05,4926 CHENNAULT ROAD,HOUSTON,HARRIS,77033.0,-95.344966,29.681655,332100.0,EMERGENCY CALL OR REQUEST FOR ASSISTANCE,NaN,

### 3. Write

In [33]:
with dw.open_remote_file(DTW_OIS_PROJECT, CLEANED_FILENAME) as w:
    print("Writing to data.world:", CLEANED_FILENAME)
    shootings.to_csv(w, index=False)

Writing to data.world: shot_civilians.csv
